# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

## Objetivos

Identify the single feature that has the strongest predictive performance for classifying crop types.

- Find the feature in the dataset that produces the best score for predicting `"crop"`.
- From this information, create a variable called `best_predictive_feature`, which:
  - Should be a `dictionary` containing the best predictive feature name as a key and the evaluation score (for the metric you chose) as the value.

In [13]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np

# Load the dataset
crops = pd.read_csv("soil_measures.csv")
print(crops.head(), print(crops.shape))


(2200, 5)
    N   P   K        ph  crop
0  90  42  43  6.502985  rice
1  85  58  41  7.038096  rice
2  60  55  44  7.840207  rice
3  74  35  40  6.980401  rice
4  78  42  42  7.628473  rice None


# Minha Solução

## Etapa 1: Verificando dados disponíveis (EDA para checar valores únicos e nulos)

In [14]:
crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB


In [15]:
crops.info()
print(crops.isna().sum())
for i in crops.columns:
    print(i)
    print(crops[i].unique())
    print(crops[i].value_counts())
    print("\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB
N       0
P       0
K       0
ph      0
crop    0
dtype: int64
N
[ 90  85  60  74  78  69  94  89  68  91  93  77  88  76  67  83  98  66
  97  84  73  92  95  99  63  62  64  82  79  65  75  71  72  70  86  61
  81  80 100  87  96  40  23  39  22  36  32  58  59  42  28  43  27  50
  25  31  26  54  57  49  46  38  35  52  44  24  29  20  56  37  51  41
  34  30  33  47  53  45  48  13   2  17  12   6  10  19  11  18  21  16
   9   1   7   8   0   3   4   5  14  15  55 105 108 118 101 106 109 117
 114 110 112 111 102 116 119 107 104 103 120 113 115 133 136 126 121 

Aparentemente, os dados já estão adequadamente cleaned, sem valores nulos e informações das colunas padronizadas! Os dados aparentam estar bem balanceados também. Podemos verificar isso com um **teste qui-quadrado de aderência**.

In [22]:
from scipy.stats import chisquare
import numpy as np

# Supondo que y é seu vetor de classes (pode ser y ou y_train)
qtd_classes = crops['crop'].value_counts().values  # Conta quantos de cada classe
qtd_classes_esperadas = [np.mean(qtd_classes)] * len(qtd_classes)  # Frequência esperada igual para todas as classes

stat, p = chisquare(qtd_classes, f_exp=qtd_classes_esperadas)
print(f"Teste Qui-quadrado de Aderência -> Estatística: {stat:.2f}, p-valor: {p:.4f}")

if p > 0.05:
    print("Não se pode rejeitar a hipótese de balanceamento entre as classes.")
else:
    print("Rejeita-se a hipótese de balanceamento entre as classes")

Teste Qui-quadrado de Aderência -> Estatística: 0.00, p-valor: 1.0000
Não se pode rejeitar a hipótese de balanceamento entre as classes.


O resultado acima é óbvio se visualizarmos que todas as classes são igualmente observadas no dataset, mas apenas para garantir... 

## Etapa 2: Verificando a feature que apresenta o melhor score para predizer o outcome de crop

Similar ao projeto 8!

O problema se trata de um problema de classificação. Será utilizado um classificador do SKLearn para avaliar as features e identificar qual apresenta maior acurácia na predição das crops!

Primeiramente definir X e y como Features e Target

In [16]:
X = crops.drop(['crop'], axis=1)
y = crops['crop']
feature_names = X.columns
print(X.shape, y.shape)
print(feature_names)
print(X.head())
print(y.head())

(2200, 4) (2200,)
Index(['N', 'P', 'K', 'ph'], dtype='object')
    N   P   K        ph
0  90  42  43  6.502985
1  85  58  41  7.038096
2  60  55  44  7.840207
3  74  35  40  6.980401
4  78  42  42  7.628473
0    rice
1    rice
2    rice
3    rice
4    rice
Name: crop, dtype: object


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

models = []
accuracies = []
f1_scores = [] #adicionado depois para comparar com o resultado do gabarito

for feature in feature_names:
    model = LogisticRegression(max_iter = 10000, multi_class='multinomial', solver='newton-cg') 
    # model = LogisticRegression(multi_class="multinomial") -> Adicionado depois para comparar com o resultado do gabarito
    model.fit(X_train[[feature]], y_train) 
    models.append(model)
    y_pred = model.predict(X_test[[feature]])
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred, average="weighted") #adicionado depois para comparar com o resultado do gabarito
    f1_scores.append(f1) #adicionado depois para comparar com o resultado do g
    print(f"Accuracy for {feature}: {accuracy}. F1 Score: {f1}") #f1_score adicionado depois para comparar com o resultado do gabarito
    accuracies.append(accuracy)

print(models)
print(accuracies)


c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy for N: 0.14545454545454545. F1 Score: 0.10408698154331626


c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy for P: 0.18863636363636363. F1 Score: 0.1281669634364753


c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy for K: 0.24772727272727274. F1 Score: 0.1897181811666794
Accuracy for ph: 0.09772727272727273. F1 Score: 0.04539052723723777
[LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg'), LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg'), LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg'), LogisticRegression(max_iter=10000, multi_class='multinomial',
                   solver='newton-cg')]
[0.14545454545454545, 0.18863636363636363, 0.24772727272727274, 0.09772727272727273]


c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [20]:
feature_scores = dict(zip(feature_names, accuracies))
print(feature_scores)

best_predictive_feature = {feature_names[accuracies.index(max(accuracies))]:max(accuracies)}
best_predictive_feature

{'N': 0.14545454545454545, 'P': 0.18863636363636363, 'K': 0.24772727272727274, 'ph': 0.09772727272727273}


{'K': 0.24772727272727274}

# Extensão

Vamos além: fazer um modelo de predição de crop baseado em múltiplas features utilizando algoritmos de árvore que busquem um bom desempenho de acurácia para classificação.

Objetivo: Avaliar utilizando SingleTreeClassification, Random Forest, Gradient Boosting, Stochastic Gradient Boosting

Para cada caso, realize um hyperparam tuning dentro de um intervalo razoável e encontre qual dos algoritmos é o mais recomendado com os dados apresentados.

In [21]:
model_results = {}


## Logistic Regression com Todas as features

In [22]:
from sklearn.model_selection import GridSearchCV

log_reg_classifier = LogisticRegression()

log_reg_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'saga', 'liblinear'],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'max_iter': [1000, 5000, 10000]
}

log_reg_grid = GridSearchCV(estimator=log_reg_classifier, param_grid=log_reg_param_grid, cv=5, n_jobs=-1)
log_reg_grid.fit(X_train, y_train)
y_pred = log_reg_grid.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


c:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
525 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1193, in fit
    solver = _ch

NameError: name 'accuracy_score' is not defined

In [ ]:
model_results['Logistic Regression Classifier'] = [log_reg_classifier, log_reg_grid.best_params_, accuracy]

### Single Tree Classifier + grid search

In [ ]:
from sklearn.tree import DecisionTreeClassifier

single_tree_class = DecisionTreeClassifier(random_state=42)
single_tree_class_params = {
    'max_depth': np.arange(2, 11),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(2, 6),
    'criterion': ['gini', 'entropy'],
    'max_features': [None, 'sqrt', 'log2']
}
single_tree_grid = GridSearchCV(estimator=single_tree_class, param_grid=single_tree_class_params, cv=10, n_jobs=-1)
single_tree_grid.fit(X_train, y_train)


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': array([2, 3, 4, 5]),
                         'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])})

In [ ]:
model_results['Decision Tree Classifier'] = [single_tree_grid.best_estimator_, single_tree_grid.best_params_], accuracy_score(y_test, single_tree_grid.best_estimator_.predict(X_test))

### Random Forest Classifier + Random Search CV (grid search demora bastante para uma grande quantidade de opções)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

random_forest_classifier = RandomForestClassifier(random_state=42)
random_forest_params = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': np.arange(2, 11),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(2, 6),
    'criterion': ['gini', 'entropy'],
    'max_features': [None, 'sqrt', 'log2']
}

random_forest_grid = RandomizedSearchCV(estimator=random_forest_classifier, n_iter=20, param_distributions=random_forest_params, cv=4, n_jobs=-1, random_state=42)
random_forest_grid.fit(X_train, y_train)

RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'min_samples_leaf': array([2, 3, 4, 5]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': [50, 100, 150, 200,
                                                         250]},
                   random_state=42)

In [ ]:
model_results['Random Forest Classifier'] = [random_forest_grid.best_estimator_, random_forest_grid.best_params_], accuracy_score(y_test, random_forest_grid.best_estimator_.predict(X_test))

### Gradient Boosting + GridSearch CV

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradient_boosting_classifier = GradientBoostingClassifier(random_state=42)
gradient_boosting_params = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': np.arange(2, 11),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(2, 6),
    'subsample': [0.6, 0.8, 1.0],
    'criterion': ['friedman_mse'],
    'max_features': [None, 'sqrt', 'log2']
}

gradient_boosting_grid = RandomizedSearchCV(estimator=gradient_boosting_classifier, n_iter=20, param_distributions=gradient_boosting_params, cv=4, n_jobs=-1, random_state=42)
gradient_boosting_grid.fit(X_train, y_train)

RandomizedSearchCV(cv=4, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['friedman_mse'],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'max_features': [None, 'sqrt', 'log2'],
                                        'min_samples_leaf': array([2, 3, 4, 5]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': [50, 100, 150, 200],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42)

In [ ]:
model_results['Gradient Boosting Classifier'] = [gradient_boosting_grid.best_estimator_, gradient_boosting_grid.best_params_], accuracy_score(y_test, gradient_boosting_grid.best_estimator_.predict(X_test))
model_results

{'Logistic Regression Classifier': [LogisticRegression(),
  {'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'newton-cg'},
  0.6909090909090909],
 'Decision Tree Classifier': ([DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=4,
                          min_samples_split=10, random_state=42),
   {'criterion': 'entropy',
    'max_depth': 8,
    'max_features': None,
    'min_samples_leaf': 4,
    'min_samples_split': 10}],
  0.7931818181818182),
 'Random Forest Classifier': ([RandomForestClassifier(criterion='entropy', max_depth=8, min_samples_leaf=2,
                          min_samples_split=10, n_estimators=150, random_state=42),
   {'n_estimators': 150,
    'min_samples_split': 10,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'max_depth': 8,
    'criterion': 'entropy'}],
  0.8068181818181818),
 'Gradient Boosting Classifier': ([GradientBoostingClassifier(learning_rate=0.01, max_depth=6, max_features='log2',
                             

Melhor modelo de classificação baseado na acurácia: Gradient Boosting > Random Forest > Decision Tree > Logistic Regression

## Solução ("Gabarito")

In [2]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")


In [ ]:
# Check for missing values
crops.isna().sum()

N       0
P       0
K       0
ph      0
crop    0
dtype: int64

In [3]:
# Check how many crops we have, i.e., multi-class target
crops.crop.unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [11]:
# Split into feature and target sets
X = crops.drop(columns="crop")
y = crops["crop"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


    N   P   K        ph
0  90  42  43  6.502985
1  85  58  41  7.038096
2  60  55  44  7.840207
3  74  35  40  6.980401
4  78  42  42  7.628473
0    rice
1    rice
2    rice
3    rice
4    rice
Name: crop, dtype: object


In [5]:
# Create a dictionary to store the model performance for each feature
feature_performance = {}
accuracy_performance = {} # Adicionado por mim para teste
# Train a logistic regression model for each feature
for feature in ["N", "P", "K", "ph"]:
    log_reg = LogisticRegression(multi_class="multinomial")
    log_reg.fit(X_train[[feature]], y_train)
    y_pred = log_reg.predict(X_test[[feature]])
    
    # Calculate F1 score, the harmonic mean of precision and recall
    # Could also use balanced_accuracy_score
    f1 = metrics.f1_score(y_test, y_pred, average="weighted")
    accuracy = metrics.accuracy_score(y_test, y_pred)
    
    accuracy_performance[feature] = accuracy # Adicionado para comparar com o meu resultado
    print(f"Accuracy for {feature}: {accuracy}")
    # Add feature-f1 score pairs to the dictionary
    feature_performance[feature] = f1
    print(f"F1-score for {feature}: {f1}")

# K produced the best F1 score
# Store in best_predictive_feature dictionary
best_predictive_feature = {"K": f'F1_score:{feature_performance["K"]}, Accuracy_score:{accuracy_performance["K"]}'}
best_predictive_feature

Accuracy for N: 0.13863636363636364
F1-score for N: 0.09149868209906838
Accuracy for P: 0.20227272727272727
F1-score for P: 0.1476194290972821
Accuracy for K: 0.2863636363636364
F1-score for K: 0.23896974566001802
Accuracy for ph: 0.09772727272727273
F1-score for ph: 0.0458225366614312


c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\User\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always us

{'K': 'F1_score:0.23896974566001802, Accuracy_score:0.2863636363636364'}

# Comparando Resultados com o Gabarito e Conclusões finais

A solução proposta como gabarito utiliza apenas o modelo de regressão logística comparando as features individualmente (que era o objetivo da solução) utilizando a métrica F1_Score, que é a média harmônica entre as métricas de precision e recall. Essa métrica normalmente é utilizada em datasets desbalanceados, no entanto, como foi provado nos resultados, o dataset é balanceado e portanto utilizar a métrica de acurácia apenas já seria suficiente e significativo.


A modelagem do gabairto apresentou como resultado que  Feature mais impactante na predição das "crops" é a "K" com F1_Score: 0.2390 e Accuracy_score: 0.2864. 

Na minha solução apresentada, ao utilizar o solver `'newton-cg'` e `n_iter=10000`, obtive resultados parecidos ("K" como sendo a feature mais impactante), porém apresentou `F1_score: 0.1897` e `Accuracy_score: 0.2478`. O Solver foi escolhido arbitrariamente, embora tenha a mesma aplicabilidade/validade no problema em questão do que quando comparado com o solver `'lbfgs'`. 

Como prova de que a variação é devida à seleção destes hiperparametros, colocou-se uma linha comentada que pode ser substituída pela inicialização do objeto de LogisticRegression na solução apresentada inicialmente.

Por fim, pode-se ressaltar ainda que os resultados do modelo de predição utilizando a regressão logística com todas as features leva à um modelo com acurácia inferior a 70%, e que a utilização de algoritmos baseados em árvore (Decision Tree, Random Forest e Gradient Boost) alcançaram acurácia entre 79% e 81%, sendo o melhor desempenho ocorrendo nos modelos Gradient Boost, seguido por Random Forest e por fim Decision Tree Classifier.


